In [ ]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import pickle
from tqdm.notebook import tqdm
%matplotlib inline

In [ ]:
raw_data = torch.load('/gan-clouds/5gaussians.pt')

In [ ]:
raw_data.keys()

In [ ]:
clouds = [raw_data['gaussians'], raw_data['WGAN_GP'], raw_data['GAN']]

In [ ]:
for cloud in clouds:
    print(cloud.shape)

In [ ]:
import mtd

In [ ]:
%time
res1 = []
trials = 20
cuda = 0

for i, cloud in enumerate(clouds):
    np.random.seed(7)
    barcs = [mtd.calc_cross_barcodes(clouds[i], clouds[0], batch_size1 = 100, batch_size2 = 1000,\
                                    cuda = cuda) for _ in range(trials)]
    res1.append(barcs)

In [ ]:
%time
res2 = []
trials = 20
cuda = 0

for i, cloud in enumerate(clouds):
    np.random.seed(7)
    barcs = [mtd.calc_barcodes(clouds[0], clouds[i], batch_size1 = 100, batch_size2 = 1000,\
                                    cuda = cuda) for _ in range(trials)]
    res2.append(barcs)

In [ ]:
#with open('gaussians.pickle', 'wb') as outfile:
#    pickle.dump((res1, res2), outfile)

In [ ]:
def get_scores(res, args_dict, trials = 10):

    scores = []

    for i in range(len(res)): 
        asum = []
        
        for exp_id, elem in enumerate(res[i]):
            asum.append(mtd.get_score(elem, **args_dict))
        scores.append(sum(asum) / len(res[i]))

    return scores

In [ ]:
scores = get_scores(res1, {'h_idx' : 1, 'kind' : 'sum_length'})

In [ ]:
descriptions = ['Gaussians', 'WGAN_GP', 'GAN']

In [ ]:
for s, d in zip(scores, descriptions):
    print(d, s)

In [ ]:
### Geometry Score
import gs

def get_rlts(X):
    N = 2500
    gamma = (1/128)/(N/5000)
    rlts = gs.rlts(X, gamma=gamma, n=N, n_threads = 10)
    
    return rlts

In [ ]:
rlts = [None] * len(clouds)

In [ ]:
%time
for i in range(len(clouds)):
    rlts[i] = get_rlts(clouds[i])

In [ ]:
for i in range(len(clouds)):
    print(1e3 * gs.geom_score(rlts[0], rlts[i]))

In [1]:
# additional experiments with IMD

from msid import msid_score
np.random.seed(7)
res = []

for i, cloud in tqdm(list(enumerate(clouds))):
    v = msid_score(clouds[0], clouds[i])
    res.append(v)

In [ ]:
plt.scatter(x = clouds[0][:,0], y = clouds[0][:,1], s = 1)
plt.scatter(x = clouds[1][:,0], y = clouds[1][:,1], s = 1)
plt.scatter(x = clouds[2][:,0], y = clouds[2][:,1], s = 1)
plt.legend(['data', 'WGAN-GP', 'GAN'], loc=1)